In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Usar GPU
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # Usar CPU
    print("GPU no disponible, se utilizará la CPU")

In [ ]:
# Input parameters
number_raster_layers = 9
number_epoch = 1000
radius_maps = "1050"

In [ ]:
# Extract training and test info

df_train = pd.read_csv('../../Data/data_train_new.csv')
df_test = pd.read_csv('../../Data/data_test_new.csv')

rows_with_na = df_test.isna().any(axis=1)
indices_na = df_test.index[rows_with_na]
df_test = df_test.drop(index=indices_na)

In [ ]:
number_training_sites = df_train.shape[0]
number_test_sites = df_test.shape[0]

print("Training sites:",number_training_sites)
print("Test sites:",number_test_sites)

In [ ]:
# Extract training/test values and sites
train_values = df_train["log_vr_total"][0:number_training_sites]
train_sites = df_train["plot_id"][0:number_training_sites]

test_values = df_test["log_vr_total"][0:number_test_sites]
test_sites = df_test["plot_id"][0:number_test_sites]

In [ ]:
# Turn the response variable into tensors
y_train = torch.tensor(np.repeat(train_values.values,4)) # We repeat each value 4 times because we included the raster and 3 rotations
y_test = torch.tensor(test_values.values) # same thing here
y_train = y_train.unsqueeze(1)
y_test = y_test.unsqueeze(1)

In [ ]:
# Estimate the minimum number of pixels per layer
min_rows_train = 100000
min_columns_train = 100000
min_rows_test = 100000
min_columns_test = 100000

for site in train_sites:
    
    # print(site)
    
    data1 = pd.read_csv('../../Data/Copernicus_maps/Bare_df/radius_' + radius_maps + '/' + site + '.csv')
    data2 = pd.read_csv('../../Data/Copernicus_maps/BuiltUp_df/radius_' + radius_maps + '/' + site + '.csv')
    
    rows_data1 = data1.shape[0]
    columns_data1 = data1.shape[1]
    
    if rows_data1 < min_rows_train:
        min_rows_train = rows_data1
        
    if columns_data1 < min_columns_train:
        min_columns_train = columns_data1

for site in test_sites:
    
    # print(site)
    
    data1 = pd.read_csv('../../Data/Copernicus_maps/Bare_df/radius_' + radius_maps + '/' + site + '.csv')
    data2 = pd.read_csv('../../Data/Copernicus_maps/BuiltUp_df/radius_' + radius_maps + '/' + site + '.csv')
    
    rows_data1 = data1.shape[0]
    columns_data1 = data1.shape[1]
    
    if rows_data1 < min_rows_test:
        min_rows_test = rows_data1
        
    if columns_data1 < min_columns_test:
        min_columns_test = columns_data1

        
print(min_rows_train)
print(min_columns_train)
print(min_rows_test)
print(min_columns_test)

number_pixels_layer = min(min_rows_train,min_rows_test,min_columns_train,min_columns_test)

if number_pixels_layer % 2 == 0:
    number_pixels_layer = number_pixels_layer - 1

print('Minimum number of pixels per (square) layer:', number_pixels_layer)

In [ ]:
def adjust_dataframe(data,number_pixels_layer):
    
    # turn nans into zeros
    data = data.fillna(0)
    
    rows_data = data.shape[0]
    columns_data = data.shape[1]
    
    # if the number of rows/columns are even we remove first row/column
    if rows_data % 2 == 0:
        data = data.drop([0])
        rows_data = data.shape[0]
    if columns_data % 2 == 0:
        data = data.drop(data.columns[0], axis=1)
        columns_data = data.shape[1]
        
    # Reduce images a matrices with number_pixels_layer x number_pixels_layer pixels
    
    while columns_data != number_pixels_layer:
        
        data = data.drop(data.columns[0], axis=1)
        columns_data = data.shape[1]
        data = data.drop(data.columns[(columns_data-1)], axis=1)
        columns_data = data.shape[1]
        
    while rows_data != number_pixels_layer:
        
        data = data.drop(data.index[[0, len(data)-1]])
        rows_data = data.shape[0]
        
    return data

In [ ]:
# To increase the number of training maps we rotated them.

def create_tensor_from_9_dataframe_layers(data1 ,data2, data3, data4, data5, data6, data7, data8, data9):
    
    # create tensor from data layer 
    tensor1 = torch.tensor(data1.values)
    tensor2 = torch.tensor(data2.values)
    tensor3 = torch.tensor(data3.values)
    tensor4 = torch.tensor(data4.values)
    tensor5 = torch.tensor(data5.values)
    tensor6 = torch.tensor(data6.values)
    tensor7 = torch.tensor(data7.values)
    tensor8 = torch.tensor(data8.values)
    tensor9 = torch.tensor(data9.values)
        
    return torch.stack((tensor1, tensor2, tensor3, tensor4, tensor5, tensor6, tensor7, tensor8, tensor9), dim=0)

def create_tensor_from_9_dataframe_layers_rotated_90_clock(data1 ,data2, data3, data4, data5, data6, data7, data8, data9):
    
    # rotate layers (90º, clockwise) 
    data1 = np.rot90(data1, k = -1)
    data2 = np.rot90(data2, k = -1)
    data3 = np.rot90(data3, k = -1)
    data4 = np.rot90(data4, k = -1)
    data5 = np.rot90(data5, k = -1)
    data6 = np.rot90(data6, k = -1)
    data7 = np.rot90(data7, k = -1)
    data8 = np.rot90(data8, k = -1)
    data9 = np.rot90(data9, k = -1)
    
    data1 = pd.DataFrame(data1.copy())
    data2 = pd.DataFrame(data2.copy())
    data3 = pd.DataFrame(data3.copy())
    data4 = pd.DataFrame(data4.copy())
    data5 = pd.DataFrame(data5.copy())
    data6 = pd.DataFrame(data6.copy())
    data7 = pd.DataFrame(data7.copy())
    data8 = pd.DataFrame(data8.copy())
    data9 = pd.DataFrame(data9.copy())
        
    return create_tensor_from_9_dataframe_layers(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)

def create_tensor_from_9_dataframe_layers_rotated_90_counter(data1 ,data2, data3, data4, data5, data6, data7, data8, data9):
    
    # rotate layers (90º, counterclockwise) 
    data1 = np.rot90(data1, k = 1)
    data2 = np.rot90(data2, k = 1)
    data3 = np.rot90(data3, k = 1)
    data4 = np.rot90(data4, k = 1)
    data5 = np.rot90(data5, k = 1)
    data6 = np.rot90(data6, k = 1)
    data7 = np.rot90(data7, k = 1)
    data8 = np.rot90(data8, k = 1)
    data9 = np.rot90(data9, k = 1)
    
    data1 = pd.DataFrame(data1.copy())
    data2 = pd.DataFrame(data2.copy())
    data3 = pd.DataFrame(data3.copy())
    data4 = pd.DataFrame(data4.copy())
    data5 = pd.DataFrame(data5.copy())
    data6 = pd.DataFrame(data6.copy())
    data7 = pd.DataFrame(data7.copy())
    data8 = pd.DataFrame(data8.copy())
    data9 = pd.DataFrame(data9.copy())
        
    return create_tensor_from_9_dataframe_layers(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)

def create_tensor_from_9_dataframe_layers_rotated_180(data1 ,data2, data3, data4, data5, data6, data7, data8, data9):
    
     # create tensor from data layer 
    data1 = np.rot90(np.rot90(data1, k = -1), k = -1)
    data2 = np.rot90(np.rot90(data2, k = -1), k = -1)
    data3 = np.rot90(np.rot90(data3, k = -1), k = -1)
    data4 = np.rot90(np.rot90(data4, k = -1), k = -1)
    data5 = np.rot90(np.rot90(data5, k = -1), k = -1)
    data6 = np.rot90(np.rot90(data6, k = -1), k = -1)
    data7 = np.rot90(np.rot90(data7, k = -1), k = -1)
    data8 = np.rot90(np.rot90(data8, k = -1), k = -1)
    data9 = np.rot90(np.rot90(data9, k = -1), k = -1)
    
    data1 = pd.DataFrame(data1.copy())
    data2 = pd.DataFrame(data2.copy())
    data3 = pd.DataFrame(data3.copy())
    data4 = pd.DataFrame(data4.copy())
    data5 = pd.DataFrame(data5.copy())
    data6 = pd.DataFrame(data6.copy())
    data7 = pd.DataFrame(data7.copy())
    data8 = pd.DataFrame(data8.copy())
    data9 = pd.DataFrame(data9.copy())
        
    return create_tensor_from_9_dataframe_layers(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)

In [ ]:
# Create a training tensor
for site in train_sites:
    
    # print(site)
    
    data1 = pd.read_csv('../../Data/Copernicus_maps/Bare_df/radius_'+ radius_maps +'/' + site + '.csv')
    data2 = pd.read_csv('../../Data/Copernicus_maps/BuiltUp_df/radius_'+ radius_maps +'/' + site + '.csv')
    data3 = pd.read_csv('../../Data/Copernicus_maps/Crops_df/radius_'+ radius_maps +'/' + site + '.csv')
    data4 = pd.read_csv('../../Data/Copernicus_maps/Grass_df/radius_'+ radius_maps +'/' + site + '.csv')
    data5 = pd.read_csv('../../Data/Copernicus_maps/MossLichen_df/radius_'+ radius_maps +'/' + site + '.csv')
    data6 = pd.read_csv('../../Data/Copernicus_maps/PermanentWater_df/radius_'+ radius_maps +'/' + site + '.csv')
    data7 = pd.read_csv('../../Data/Copernicus_maps/SeasonalWater_df/radius_'+ radius_maps +'/' + site + '.csv')
    data8 = pd.read_csv('../../Data/Copernicus_maps/Shrub_df/radius_'+ radius_maps +'/' + site + '.csv')
    data9 = pd.read_csv('../../Data/Copernicus_maps/Tree_df/radius_'+ radius_maps +'/' + site + '.csv')
    
    data1 = adjust_dataframe(data1,number_pixels_layer)
    data2 = adjust_dataframe(data2,number_pixels_layer)
    data3 = adjust_dataframe(data3,number_pixels_layer)
    data4 = adjust_dataframe(data4,number_pixels_layer)
    data5 = adjust_dataframe(data5,number_pixels_layer)
    data6 = adjust_dataframe(data6,number_pixels_layer)
    data7 = adjust_dataframe(data7,number_pixels_layer)
    data8 = adjust_dataframe(data8,number_pixels_layer)
    data9 = adjust_dataframe(data9,number_pixels_layer)

    # Create tensor from layers
    tensor_9capas = create_tensor_from_9_dataframe_layers(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)
    
    # Create tensor from layers rotated 90º clockwise
    tensor_9capas_rotated_90_clock = create_tensor_from_9_dataframe_layers_rotated_90_clock(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)
    
    # Create tensor from layers rotated 90º counterclockwise
    tensor_9capas_rotated_90_counter = create_tensor_from_9_dataframe_layers_rotated_90_counter(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)
    
    # Create tensor from layers rotated 180º
    tensor_9capas_rotated_180 = create_tensor_from_9_dataframe_layers_rotated_180(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)
    
    # print(tensor_9capas.shape)

    tensor_9capas = tensor_9capas.unsqueeze(0)
    tensor_9capas_rotated_90_clock = tensor_9capas_rotated_90_clock.unsqueeze(0)
    tensor_9capas_rotated_90_counter = tensor_9capas_rotated_90_counter.unsqueeze(0)
    tensor_9capas_rotated_180 = tensor_9capas_rotated_180.unsqueeze(0)
    
    # print(tensor_9capas.shape)
    
    if site == train_sites.iloc[0]:
        
        training_tensor = tensor_9capas
        training_tensor = torch.cat((training_tensor, tensor_9capas_rotated_90_clock), dim=0)
        training_tensor = torch.cat((training_tensor, tensor_9capas_rotated_90_counter), dim=0)
        training_tensor = torch.cat((training_tensor, tensor_9capas_rotated_180), dim=0)
        
    else:
        
        training_tensor = torch.cat((training_tensor, tensor_9capas), dim=0)
        training_tensor = torch.cat((training_tensor, tensor_9capas_rotated_90_clock), dim=0)
        training_tensor = torch.cat((training_tensor, tensor_9capas_rotated_90_counter), dim=0)
        training_tensor = torch.cat((training_tensor, tensor_9capas_rotated_180), dim=0)
        
    
print(training_tensor.shape)

In [ ]:
# Create a test tensor
for site in test_sites:
    
    # print(site)
    
    data1 = pd.read_csv('../../Data/Copernicus_maps/Bare_df/radius_'+ radius_maps +'/' + site + '.csv')
    data2 = pd.read_csv('../../Data/Copernicus_maps/BuiltUp_df/radius_'+ radius_maps +'/' + site + '.csv')
    data3 = pd.read_csv('../../Data/Copernicus_maps/Crops_df/radius_'+ radius_maps +'/' + site + '.csv')
    data4 = pd.read_csv('../../Data/Copernicus_maps/Grass_df/radius_'+ radius_maps +'/' + site + '.csv')
    data5 = pd.read_csv('../../Data/Copernicus_maps/MossLichen_df/radius_'+ radius_maps +'/' + site + '.csv')
    data6 = pd.read_csv('../../Data/Copernicus_maps/PermanentWater_df/radius_'+ radius_maps +'/' + site + '.csv')
    data7 = pd.read_csv('../../Data/Copernicus_maps/SeasonalWater_df/radius_'+ radius_maps +'/' + site + '.csv')
    data8 = pd.read_csv('../../Data/Copernicus_maps/Shrub_df/radius_'+ radius_maps +'/' + site + '.csv')
    data9 = pd.read_csv('../../Data/Copernicus_maps/Tree_df/radius_'+ radius_maps +'/' + site + '.csv')
    
    data1 = adjust_dataframe(data1,number_pixels_layer)
    data2 = adjust_dataframe(data2,number_pixels_layer)
    data3 = adjust_dataframe(data3,number_pixels_layer)
    data4 = adjust_dataframe(data4,number_pixels_layer)
    data5 = adjust_dataframe(data5,number_pixels_layer)
    data6 = adjust_dataframe(data6,number_pixels_layer)
    data7 = adjust_dataframe(data7,number_pixels_layer)
    data8 = adjust_dataframe(data8,number_pixels_layer)
    data9 = adjust_dataframe(data9,number_pixels_layer)

    # Create tensor from layers
    tensor_9capas = create_tensor_from_9_dataframe_layers(data1 ,data2, data3, data4, data5, data6, data7, data8, data9)

    tensor_9capas = tensor_9capas.unsqueeze(0)
    
    if site == test_sites.iloc[0]:
        
        test_tensor = tensor_9capas
        
    else:
        
        test_tensor = torch.cat((test_tensor, tensor_9capas), dim=0)
        

print(test_tensor.shape)

In [ ]:
# Model definition
dropout_rate = 0.5

class CNNRegressor(nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv2d(number_raster_layers, 16, kernel_size=3, stride=1, padding=1)
        self.selu1 = nn.SELU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.selu2 = nn.SELU()
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.selu3 = nn.SELU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.selu4 = nn.SELU()
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.selu5 = nn.SELU()
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.selu6 = nn.SELU()
        #self.dropout = nn.Dropout(dropout_rate)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(512 * number_pixels_layer * number_pixels_layer, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.selu1(x)
        x = self.conv2(x)
        x = self.selu2(x)
        x = self.conv3(x)
        x = self.selu3(x)
        x = self.conv4(x)
        x = self.selu4(x)
        x = self.conv5(x)
        x = self.selu5(x)
        x = self.conv6(x)
        x = self.selu6(x)
        x = self.flatten(x)
        # x = self.dropout(x)
        x = self.fc(x)
        return x

model = CNNRegressor()

In [ ]:
# DO NOT RUN: COMPUTERS MEMORY RUNS OUT
## Check GPU availability
#if torch.cuda.is_available():
#    device = torch.device("cuda")
#    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
#else:
#    device = torch.device("cpu")
#    print("GPU no disponible, se utilizará la CPU")

## Create a model and send it to GPUs
#model = CNNRegressor().to(device)

## Send data to GPUs
#training_tensor = training_tensor.to(device)
#y_train = y_train.to(device)

In [ ]:
# Loss function and optimization
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
import torch.nn.functional as F

for epoch in range(number_epoch):
    optimizer.zero_grad()
    outputs = model(training_tensor.to(torch.float))
    loss = criterion(outputs, y_train.to(torch.float))
    
    # L2 regularization
    l2_lambda = 0.01  # L2 regularization factor
    l2_regularization = torch.tensor(0.)  # init L2 regularization factor

    for param in model.parameters():
        l2_regularization += torch.norm(param, 9)  # add L2 norm of each parameter

    loss += l2_lambda * l2_regularization  # add L2 reg to total loss

    
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{number_epoch}], Loss: {loss.item()}')


In [ ]:
# Save the fitted model
path_model = '../../Data/Calibrated_models/global_regressor_V0.pth'
torch.save(model.state_dict(), path_model)


